# Configuración de Ollama y Jupyter Lab - Primera consulta a un modelo - JKGzenna

Ejemplo para resumir una página web y utilizar un modelo de código abierto con Ollama que se ejecute localmente a través de Ollama API en lugar de OpenAI, Anthropic o cualquier otra plataforma de pago

Se puede utilizar esta técnica para todos los proyectos posteriores si se prefiere no utilizar API de pago (closed source).

**Beneficios:**
1. Sin cargos por API: código abierto
2. Los datos no salen de su ordenador

**Desventajas:**
1. Tiene mucha menos potencia (parámetros) que el modelo Frontier de cualquier empresa de pago

## Resumen de la instalación de Ollama

¡Hemos levantado Ollama en stack de contendores junto con jupyter en un misma net de docker, y con retorno al host anfitrión, para que puedas trabajar con Ollama y Jupyter como si los tuvieras en local, pero estando contenerizados, de esa manera no 'ensuciamos' nuestros equipos instalando dependencias y programas de manera local, mucho mejor en un contenedor al que le pasemos nuestros datos con un volumen.

Una vez que hayas terminado, el servidor Ollama ya debería estar ejecutándose localmente en tu máquina, como si hubieras hecho la instalación en windows/linux/mac de Ollama.

Si entras en: `http://localhost:11434/`

Debería ver el mensaje: `Ollama is running`

Cada vez que hagamos un uso de algun modelo de ollama que este pulleado en nuestro equipo automaticamente se levantara si esta pulleado, si no esta pulleado dará error, se debe hacer pull de el en terminal, al pasar un par de minutos sin consultas lo baja de nuevo, en ollama desde terminal podemos controlar los modelos igual que si fueran contenedores con los siguientes comandos:

- `ollama pull`
- `ollama list`
- `ollama ps` (para ver modelos activos en ejecución)
- `ollama rm <modelo>` (para eliminar modelos pulleados de nuestro equipo)
- `ollama run <modelo>` (solo es necesario si queremos consumirlo desde terminal de manera explicita y hacer consultas desde allí, si no jupyter los levantará y bajará si están pulleados)
- `ollama stop <modelo>` (pocas veces apagaremos a mano un modelo, ya que jupyter se encargará de bajarlos cuando no haya consultas, y si somos nosotros los que levantamos el modelo desde terminal, al terminar de hablar con el podemos salir y cerrarlo con `/bye`, en caso de que cerremos la ventana y el modelo se quede en ejecución, por eso o cualquier otro motivo, tendremos que bajarlo con este comando)

Si se ha realizado instalación con contenedores de docker se puede ver también en esta URL [http://host.docker.internal:11434/](http://host.docker.internal:11434/), a la cual es necesario apuntar desde Jupyter cuando llamemos al API de ollama en caso de estar ejecutando Ollama en contenedor como podemos ver en esta issue de GitHub: https://github.com/ollama/ollama/issues/3200

Después de levantar este stack con el `docker-compose.yml` debes abrir un terminal aquí en Jupyter haciendo click en `File` y después en `New Console For Notebook` a ejecutar el siguiente comando para instalar las dependencias necesarias, si necesitas alguna más sólo debes ejecutar el comando `!pip install` o `pip install`, al ejecutar este comando nos avisará de que es más recomendable hacerlo en un entorno virtualizado con python (pipenv), pero no vamos a hacerlo así porque no estamos en el sistema de la máquina anfitriona, donde sin duda prepararíamos un entorno virtualizado clásico con pipenv,  o bien con anaconda, pero todo eso "ensucia" nuestro sistema local anfitrión, pero como estamos dentro de un contenedor de docker creado específicamente para esta tarea, realizaremos instalación global y como root de las dependencias, sin utilizar un entorno virtualizado (pipenv o conda), ya que este contenedor es específico para esta tarea de Jupyter, pudiendo destruirlo cuando deseemos, ya que los datos con los que trabjaremos se almacenan de manera segura en un volumen de datos de docker conectado al sistema local anfitrión.

`!pip install -r requirements.txt`

In [ ]:
# Imports necesarios para la prueba del API de Ollama

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [ ]:
# Constantes

# Si hemos levantado Ollama en un contendor de Docker, debemos hacer la llamada a 'host.docker.internal' u 'ollama',
# en lugar de a 'localhost', de lo contrario obtendremos error del API al estar trabajando contra 'localhost'

# Utilizando las DNS que nos crea Docker al instalarse en el '../../../etc/hosts', esta dirección funciona en el navegador
#OLLAMA_API = "http://host.docker.internal:11434/api/chat"

# # Utilizando el nombre del servicio que hemos creado en Docker para la imagen de Ollama, esta dirección no funciona en el navegador, solo con el API de Ollama,
# ya que no es una ruta expuesta, sino un servicio interno de docker que Jupyter es capaz de ver porque está en el mismo stack y red de Docker que Ollama
OLLAMA_API = "http://ollama:11434/api/chat"

# Incluimos la variable cabeceras para un mejor manejo de la solicitud y no sobrecargar el post que vamos a realizar más adelante
HEADERS = {"Content-Type": "application/json"}

# Elegimos un modelo opensource de Ollama'
MODEL = "gemma3n"

In [ ]:
# Crea una lista de mensajes utilizando el mismo formato que usamos para OpenAI, Anthropic o cualquier otro modelo Frontier

messages = [
    {"role": "user", "content": "Dame la receta de la tortilla de patata"}
]

In [ ]:
# Preparamos el objeto payload para pasarle directamente el nombre 'payload' y no todo lo que contiene en el siguiente post que vamos a realizar, así no sobrecargamos dicho post
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [ ]:
# Pasamos como parametros del request el API de Ollama, la varibale con las cabeceras y el objeto con el payload que hemos preparado en un post que nos devolvera la respuesta
response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
# Respuesta RAW sin formatear
# Respuesta en json normal
#print(response.json()['message']['content'])
# Respuesta formateada en Markdown
reply = (response.json()['message']['content'])
display(Markdown(reply))